# Introduction

Sudoku is a logic-based puzzle that originated in Japan.  It is appealing to people because the level of difficulty can be chosen by the user.  The levels of difficulty range from super easy to difficult.  Acceptance grew rapidly once it was introduced outside of Japan, partially because upon a quick look at a Sudoku puzzle, one can easily confuse it with a crossword puzzle (see figure 1).	

A typical Sudoku puzzle consists of a 9x9 grid that features the numbers 1-9.  As shown in figure 1, there is a main box around the 9x9 grid, but there are mini boxes of size 3x3, as seen in the grid.	

In order to win, a user must follow specific rules.  The main rules are:
* The user can only use the numbers 1 to 9.
* Every mini box (grid size 3x3) can only contain the numbers 1 to 9.
* Every mini box cannot have duplicate numbers.	
* Every vertical column can only contain the numbers 1 to 9.
* Every vertical column cannot have duplicate numbers.	
* Every horizontal row can only contain the numbers 1 to 9.
* Every horizontal row cannot have duplicate numbers.

Failure to follow the rules will result in duplicate numbers in either a mini box, a vertical column, or a horizontal row.  If the rules are followed, then the Sudoku puzzle is solved.	


# Proposed Approach	

My proposed approach is to use the Sudoku rules mentioned in the introduction.  Looking at all the rules at the same time can be stressing, but if I decompose the problems into smaller ones, then the problem is approachable as follows:	

Let $\textbf{ψ}$ represent all the columns, such that $\textbf{ψ_i}$ represents the $\textbf{i^{th}}$ column	
Let $\textbf{ϕ}$ represent all the rows, such that $\textbf{ϕ_j}$ represents the $\textbf{j^{th}}$ row	
Let $\textbf{φ}$ represent all the mini boxes, such that $\textbf{φ_k}$ represents the $\textbf{k^{th}}$ mini box	

Having assigned the columns, rows, and mini boxes to symbols, I can now enforce the rules presented in the introduction.  To verify, for example, that $\textbf{ψ_1}$ holds, I would need to verify that for every number in $\textbf{ψ_1}$ that there are no duplicates. I would then need to perform the same test on $\textbf{ψ_2…ψ_9}$.  After performing that test, $\textbf{ψ}$ will hold if and only if  $\textbf{ψ_1∧ψ_2∧ψ_3∧ψ_4∧ψ_5∧ψ_6∧ψ_7∧ψ_8∧ψ_9}$ holds.  The same will hold true for $\textbf{ϕ}$ (all the rows) and for $\textbf{φ}$ (all the mini boxes).  

Following this logic, if $\textbf{ψ∧ϕ∧φ}$ holds, then all the rules have been followed and the user has solve the Sudoku puzzle.

While I have previously completed a Sudoku solver using dynamic programing, this time I plan on using a satisfiability module theory (SMT) solver, specifically the Z3 SMT solver.  This will allow me to apply the knowledge learned from class and to learn to program in the Z3 environment.	


$\Phi$ $\Psi$  
$$\phi_1$$

### Required Imports

In [1]:
from z3 import *
from datetime import datetime

### Configuration

In [2]:
size = 9

# Restraints (from introduction)

As a recap, here are the restraints that my Sudoku Z3 solver requires

* <span style="color:green">Every mini box (grid size 3x3) can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 1</span>)
* Every mini box cannot have duplicate numbers.  
* <span style="color:green">Every vertical column can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 2</span>)
* Every vertical column cannot have duplicate numbers.  
* <span style="color:green">Every horizontal row can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 3</span>)
* Every horizontal row cannot have duplicate numbers.  
* <span style="color:green">Every mini box can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 4</span>)  
* Every box cannot have duplicate numbers.  

### Solution

We first set the solution to restrict every element (shown in <span style="color:green">green above</span>).

We'll first create integer references for all locations, $L$, in the form $L_{ij}$ where $i$ refers to the rows and $j$ refers to the columns.

In [3]:
L = []
for row in range(size):
    L.append( [Int(f'L_{row}{col}') for col in range(size)] )

Next, we create the first restraint: a conjunction for every location ($L_{ij}$) where $1 \leq L_{ij} \leq 9$, then we check the constraints, such that  
`constraint_1` = 
$$\begin{align}
&L_{00} \wedge L_{01} \wedge L_{02} \wedge L_{03} \wedge L_{04} \wedge L_{05} \wedge L_{06} \wedge L_{07} \wedge L_{08} \wedge \\
&L_{10} \wedge L_{11} \wedge L_{12} \wedge L_{13} \wedge L_{14} \wedge L_{15} \wedge L_{16} \wedge L_{17} \wedge L_{18} \wedge \\
&L_{20} \wedge L_{21} \wedge L_{22} \wedge L_{23} \wedge L_{24} \wedge L_{25} \wedge L_{26} \wedge L_{27} \wedge L_{28} \wedge \\
&L_{30} \wedge L_{31} \wedge L_{32} \wedge L_{33} \wedge L_{34} \wedge L_{35} \wedge L_{36} \wedge L_{37} \wedge L_{38} \wedge \\
&L_{40} \wedge L_{41} \wedge L_{42} \wedge L_{43} \wedge L_{44} \wedge L_{45} \wedge L_{46} \wedge L_{47} \wedge L_{48} \wedge \\
&L_{50} \wedge L_{51} \wedge L_{52} \wedge L_{53} \wedge L_{54} \wedge L_{55} \wedge L_{56} \wedge L_{57} \wedge L_{58} \wedge \\
&L_{60} \wedge L_{61} \wedge L_{62} \wedge L_{63} \wedge L_{64} \wedge L_{65} \wedge L_{66} \wedge L_{67} \wedge L_{68} \wedge \\
&L_{70} \wedge L_{71} \wedge L_{72} \wedge L_{73} \wedge L_{74} \wedge L_{75} \wedge L_{76} \wedge L_{77} \wedge L_{78} \wedge \\
&L_{80} \wedge L_{81} \wedge L_{82} \wedge L_{83} \wedge L_{84} \wedge L_{85} \wedge L_{86} \wedge L_{87} \wedge L_{88}
\end{align}$$

In [4]:
constraint_1 = [ And( L[row][col] >= 1, L[row][col] <= 9)  for col in range(size) for row in range(size) ]

We now create the second restraint: every vertical column cannot have duplicate numbers, meaning that each vertical column can only contain distinct values. To accomplish this, we create an array of arrays, where each inner array represents a conjunction of each column, such as  
`constraint_2` =  
$$
\text{Column 1}: L_{00} \wedge L_{10} \wedge \cdots \wedge L_{70} \wedge L_{80} \\
\text{Column 2}: L_{01} \wedge L_{11} \wedge \cdots \wedge L_{71} \wedge L_{81} \\
\vdots \hspace{3.5cm} \vdots \hspace{1.5cm} \\
\text{Column 8}: L_{07} \wedge L_{17} \wedge \cdots \wedge L_{77} \wedge L_{87} \\
\text{Column 9}: L_{08} \wedge L_{18} \wedge \cdots \wedge L_{78} \wedge L_{88} \\
$$

In [5]:
constraint_2 = []
for col in range(size):
    constraint_2.append( Distinct( [L[row][col] for row in range(size)] ) )

We now create the third restraint: every horizontal row cannot have duplicate numbers, meaning that each horizontal row can only contain distinct values. To accomplish this, we create an array of arrays, where each inner array represents a conjunction of each row, such as  
`constraint_3` =  
$$
\text{Row 1}: L_{00} \wedge L_{01} \wedge \cdots \wedge L_{07} \wedge L_{08} \\
\text{Row 2}: L_{10} \wedge L_{11} \wedge \cdots \wedge L_{17} \wedge L_{18} \\
\vdots \hspace{3.25cm} \vdots \hspace{1.90cm} \\
\text{Row 8}: L_{70} \wedge L_{71} \wedge \cdots \wedge L_{77} \wedge L_{78} \\
\text{Row 9}: L_{88} \wedge L_{88} \wedge \cdots \wedge L_{87} \wedge L_{88} \\
$$

In [6]:
constraint_3 = []
for row in range(size):
    constraint_3.append( Distinct( [L[row][col] for col in range(size)] ) )

We now create the fourth restraint: every mini box cannot have duplicate numbers, meaning that every mini box can only contain distinct values.  To accomplish this, we create helper a helper function, that given an $ij$ location for a mini box, will return a distinct mini box, such as
`constraint_4` =  

<img src="picture.jpg" alt="Paris" width="500px" class="center">

In [7]:
def get_mini_grid(array, i, j, size):
    return [ L[i * size + y][j * size + x] for y in range(size) for x in range(size) ]

mini_size = size // 3

constraint_4 = [ Distinct( get_mini_grid(L, y, x, 3) )  for y in range(3) for x in range(3) ]  

### Create The Solver

Now we create the solver and add the restraints

In [8]:
s = Solver()
s.add(constraint_1)
s.add(constraint_2)
s.add(constraint_3)
s.add(constraint_4)

Copy the Sudoku puzzle corresponding to every location in $L_{ij}$ and add them as constraint.  If a number is added, then it's a literal, otherwise it's left open for the Z3 solver to solve.

In [9]:
# puzzle = (
#     (0,7,2,0,0,0,1,0,0),
#     (9,0,4,0,0,0,0,0,6),
#     (0,0,0,0,7,5,0,0,2),
#     (0,3,6,0,0,4,0,0,0),
#     (4,0,0,0,9,0,0,0,3),
#     (0,0,0,3,0,0,6,1,0),
#     (3,0,0,9,8,0,0,0,0),
#     (7,0,0,0,0,0,3,0,9),
#     (0,0,8,0,0,0,2,4,0)
# )

puzzle = (
    (0,0,0,0,0,2,8,0,3),
    (0,0,0,0,0,0,0,1,0),
    (5,0,9,3,1,0,0,2,4),
    (7,0,0,0,5,3,0,0,0),
    (0,0,0,7,0,9,0,0,0),
    (0,0,0,4,2,0,0,0,1),
    (9,1,0,0,3,7,5,0,8),
    (0,4,0,0,0,0,0,0,0),
    (8,0,3,9,0,0,0,0,0)
)

puzzle = (
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,1,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,9,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
)

# constraint_5 = [ If(puzzle[i][j] == 0, True, L[i][j] == puzzle[i][j]) for i in range(9) for j in range(9) ]
constraint_5 = [L[i][j] == puzzle[i][j] for i in range(size) for j in range(size) if puzzle[i][j]]
s.add(constraint_5)

Finally we check if the model is SAT or UNSAT.  If the model is UNSAT, then we mention it; otherwise, we display the solution.

In [10]:
def get_grid(sol, as_string=True):
    s = [' '.join(str(_) for _ in sol[row]) for row in range(size)]
    s = [f'{_[:5]} | {_[6:11]} | {_[12:]}' for _ in s]
    s = [*s[:3], "-"*22, *s[3:6], "-"*22, *s[6:]]
    return '\n'.join(s) if as_string else s

def side_by_side(grid1,grid2, spaces=8):
    gs1, gs2 = get_grid(grid1), get_grid(grid2)
    s = [f'{s1}{" "*8}{s2}' for (s1, s2) in zip(gs1.split('\n'), gs2.split('\n'))]
    return '\n'.join(s)

def exec_time(start, end):
    return str(end - start)

def get_solution(model, L):
    return [ [ model.eval(L[i][j]) for j in range(size) ] for i in range(size) ]

In [11]:
start = datetime.now()
status = str(s.check())
end = datetime.now()

print('Execution time:', exec_time(start, end), '\n')

solution = 'Model is UNSAT' if status == 'unset' else get_solution(s.model(), L)

if status == 'unsat':
    print(solution)
else:
    print(side_by_side(puzzle, solution).replace('0', ' '))

Execution time: 0:00:25.095342 

      |       |              7 5 8 | 4 1 9 | 6 2 3
      |       |              1 9 2 | 3 8 6 | 7 4 5
      |       |   1          6 4 3 | 2 7 5 | 9 1 8
----------------------        ----------------------
      |       |              9 8 6 | 5 3 2 | 1 7 4
      |       |              2 3 5 | 7 4 1 | 8 9 6
      |       |              4 7 1 | 6 9 8 | 3 5 2
----------------------        ----------------------
      |       |              8 6 9 | 1 2 4 | 5 3 7
      | 9     |              5 2 7 | 9 6 3 | 4 8 1
      |       |              3 1 4 | 8 5 7 | 2 6 9


In [12]:
[ [ model.eval(L[i][j]).as_long() for j in range(size) ] for i in range(size) ]

NameError: name 'model' is not defined